In [ ]:
!pip install tpot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11938 sha256=06cc01dc43f6a4ce06904cd20fb90f5699ad7eada66818db3b018fd043d88086
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import brier_score_loss
from tpot import TPOTClassifier

import pandas as pd


train = pd.read_csv("Train_Split.csv")

train = train.drop("variance", axis=1)

X = train.iloc[:, :-1]
y = train.iloc[:, -1:]

automl = TPOTClassifier(generations=5, population_size=50, scoring='f1', verbosity=2, random_state=1, n_jobs=-1)
automl.fit(X, y)

automl.export('tpot_data.py')

model = automl.fitted_pipeline_

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model without calibration
model.fit(X_train, y_train)
uncalibrated_probs = model.predict_proba(X_valid)[:, 1]
uncalibrated_brier = brier_score_loss(y_valid, uncalibrated_probs)

# Calibrate using sigmoid
calibrated_model_sigmoid = CalibratedClassifierCV(model, method='sigmoid', cv='prefit')
calibrated_model_sigmoid.fit(X_train, y_train)
sigmoid_probs = calibrated_model_sigmoid.predict_proba(X_valid)[:, 1]
sigmoid_brier = brier_score_loss(y_valid, sigmoid_probs)

# Calibrate using isotonic
calibrated_model_isotonic = CalibratedClassifierCV(model, method='isotonic', cv='prefit')
calibrated_model_isotonic.fit(X_train, y_train)
isotonic_probs = calibrated_model_isotonic.predict_proba(X_valid)[:, 1]
isotonic_brier = brier_score_loss(y_valid, isotonic_probs)

# Compare Brier scores
print(f'Uncalibrated Brier Score: {uncalibrated_brier}')
print(f'Sigmoid Calibrated Brier Score: {sigmoid_brier}')
print(f'Isotonic Calibrated Brier Score: {isotonic_brier}')

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9418686764045721

Generation 2 - Current best internal CV score: 0.9469957413897889

Generation 3 - Current best internal CV score: 0.9508491133202375

Generation 4 - Current best internal CV score: 0.9508491133202375

Generation 5 - Current best internal CV score: 0.9508491133202375

Best pipeline: GradientBoostingClassifier(ZeroCount(input_matrix), learning_rate=0.5, max_depth=6, max_features=0.8500000000000001, min_samples_leaf=3, min_samples_split=9, n_estimators=100, subsample=0.9500000000000001)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Uncalibrated Brier Score: 0.06394867305524699
Sigmoid Calibrated Brier Score: 0.052990113121163064
Isotonic Calibrated Brier Score: 0.045463784642087356
